<a href="https://colab.research.google.com/github/Erick-M-8/Arduino/blob/main/Reporte_Eficiencia_de_canales_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar datos y librerias

In [ ]:
#Librerias
import pandas as pd

#Carga del dataset
dataset = pd.read_csv('/content/resultados_prestamo_nomina_2411.csv')
dataset.head(10)

,CLIENTE,PT,SEGMENTO,CANAL,FECHA_SOLICITUD,TOQUE,SOLICITUD,ORIGINACION,PRODUCTO,ID_ST,CREDITO_SOLICITADO,ARRANGEMENT_ID,MONTO,TENENCIA,RN
0,101099,Piloto,Empleados,SMS,2024-12-01 00:00:00.0000000,2.0,6.400007e+11,Sucursal,Préstamo Directo Nómina,BC,41153.76,NaN,NaN,4.0,1
1,6988810,Testigo,Empleados,Testigo,2024-10-31 00:00:00.0000000,NaN,6.400007e+11,Sucursal,Préstamo Directo Nómina,RT,65182.72,NaN,NaN,3.0,1
2,8542992,Piloto,Empleados,"SMS, WA",2024-11-25 00:00:00.0000000,2.0,6.400007e+11,Sucursal,Préstamo Directo Nómina,RT,75407.76,NaN,NaN,4.0,1
3,11945659,Piloto,Empleados,"SMS, EMAIL",2024-11-07 00:00:00.0000000,1.0,8.500016e+11,Sucursal,Tarjeta De Credito Grupo Coppel,AP,22500.00,225094113.0,NaN,3.0,1
4,15435760,Piloto,Empleados,"SMS, EMAIL",2024-12-03 00:00:00.0000000,2.0,6.400007e+11,Sucursal,Préstamo Directo Nómina,RT,73249.64,NaN,NaN,3.0,1
5,19550126,Piloto,Empleados,"SMS, EMAIL, WA",2024-11-27 00:00:00.0000000,2.0,7.800024e+11,Sucursal,Anticipo De Nomina,PC,NaN,NaN,NaN,3.0,1
6,20862066,Piloto,Empleados,"SMS, EMAIL",2024-11-22 00:00:00.0000000,2.0,7.800024e+11,Sucursal,Anticipo De Nomina,AP,1400.00,225548250.0,NaN,4.0,1
7,25430162,Piloto,Empleados,"SMS, EMAIL",2024-11-06 00:00:00.0000000,1.0,8.500016e+11,Sucursal,Tarjeta De Credito Grupo Coppel,RT,0.00,NaN,NaN,3.0,1
8,25683621,Piloto,Nómina Shadow,"SMS, EMAIL",2024-11-10 00:00:00.0000000,1.0,6.400007e+11,Sucursal,Préstamo Directo Nómina,RT,286230.08,NaN,NaN,3.0,1
9,26952590,Testigo,Empleados,Testigo,2024-11-26 00:00:00.0000000,2.0,6.400007e+11,One Clic,Préstamo Directo Nómina,AP,68600.00,225656855.0,4260.0,5.0,1


Canal SMS

In [ ]:
def Canal_SMS(toque):
  #Filtro para clientes contactados por SMS
  mask_sms = dataset['CANAL'] == "SMS"
  filtro_sms = dataset[mask_sms]

  #Numero de contactados por SMS
  #print("Cantidad de contactados por SMS:", len(filtro_sms.CLIENTE.unique()))
  SMS_1 = len(filtro_sms.CLIENTE.unique())

  #Tenecia por cliente
  Tabla_tenencia = filtro_sms.loc[:,['CLIENTE','TENENCIA']]
  Tabla_tenencia = Tabla_tenencia.drop_duplicates(subset='CLIENTE')
  #print(len(Tabla_tenencia),"=",SMS_1)

  #Suma de tenencia por cliente
  Suma_tenencia = Tabla_tenencia['TENENCIA'].sum()
  SMS_8 = Suma_tenencia

  #Vinculación pre campaña
  SMS_9 = round((Suma_tenencia / len(Tabla_tenencia)),3)
  #print("Vinculación pre campaña",SMS_9)

  #Contactados por SMS que ralizarón una solicitud
  mask_solic_no_nula = filtro_sms['SOLICITUD'].notnull()
  if(toque == 1):
    mask_toque_no_nulo = filtro_sms['TOQUE'].notnull()
  if(toque == 2):
    mask_toque_no_nulo = filtro_sms['TOQUE'] == 1
  if(toque == 3):
    mask_toque_no_nulo = filtro_sms['TOQUE'] == 2
  filtro_sms = filtro_sms[mask_solic_no_nula & mask_toque_no_nulo]
  #print("Cantidad de solicitudes únicas por SMS que se realizarón:", len(filtro_sms.SOLICITUD.unique()))
  SMS_2 = len(filtro_sms.SOLICITUD.unique())

  #Clientes contactados por SMS que realizarón una solicitud, aperturaron un producto
  mask_ap_other = (filtro_sms['ID_ST'] == "AP") & (filtro_sms['PRODUCTO']!= "Préstamo Directo Nómina") & (filtro_sms['PRODUCTO'].notnull())
  #print("Cantidad de contactados por SMS que realizarón una solicitud y aperturaron otro producto:", len(filtro_sms[mask_ap_other]))
  SMS_3 = len(filtro_sms[mask_ap_other])

  #Clientes contactados por SMS que realizarón una solicitud, aperturaron un prestamo directo de nomina
  mask_ap_pres = (filtro_sms['ID_ST'] == "AP") & (filtro_sms['PRODUCTO'] == "Préstamo Directo Nómina")
  filtro_sms = filtro_sms[mask_ap_pres]
  #print("Cantidad de contactados por SMS que realizarón una solicitud y aperturaron un prestamo directo de nomina:", len(filtro_sms))
  SMS_4 = len(filtro_sms)

  #Sumatoria de crédito solicitado:
  #print("Sumatoria de crédito solicitado:", filtro_sms['CREDITO_SOLICITADO'].sum())
  SMS_5 = filtro_sms['CREDITO_SOLICITADO'].sum()

  #Tasa de solicitud
  SMS_6 = round((SMS_2 / SMS_1 * 100),2)

  #Tasa de exito
  SMS_7 = round((SMS_4 / SMS_1 * 100),2)

  #Vinculación post campaña
  SMS_10 = round(((Suma_tenencia + SMS_3 + SMS_4)/ len(Tabla_tenencia)),3)
  #print("Vinculación post campaña",SMS_10)

  return (SMS_1, SMS_2, SMS_3, SMS_4, SMS_5, SMS_6, SMS_7, SMS_8, SMS_9, SMS_10)

Canal SMS y WA

In [ ]:
def Canal_SMS_WA(toque):
  #Filtro para clientes contactados por SMS, WA
  mask_sms_wa = dataset['CANAL'] == "SMS, WA"
  filtro_sms_wa = dataset[mask_sms_wa]

  #Numero de contactados por SMS, WA
  #print("Cantidad de contactados por SMS, WA:", len(filtro_sms_wa.CLIENTE.unique()))
  SMS_WA_1 = len(filtro_sms_wa.CLIENTE.unique())

  #Tenecia por cliente
  Tabla_tenencia = filtro_sms_wa.loc[:,['CLIENTE','TENENCIA']]
  Tabla_tenencia = Tabla_tenencia.drop_duplicates(subset='CLIENTE')

  #Suma de tenencia por cliente
  Suma_tenencia = Tabla_tenencia['TENENCIA'].sum()
  SMS_WA_8 = Suma_tenencia

  #Vinculación pre campaña
  SMS_WA_9 = round((Suma_tenencia / len(Tabla_tenencia)),3)
  #print("Vinculación pre campaña",SMS_WA_9)

  #Contactados por SMS, WA que ralizarón una solicitud
  mask_solic_no_nula = filtro_sms_wa['SOLICITUD'].notnull()
  if(toque == 1):
    mask_toque_no_nulo = filtro_sms_wa['TOQUE'].notnull()
  if(toque == 2):
    mask_toque_no_nulo = filtro_sms_wa['TOQUE'] == 1
  if(toque == 3):
    mask_toque_no_nulo = filtro_sms_wa['TOQUE'] == 2
  filtro_sms_wa = filtro_sms_wa[mask_solic_no_nula & mask_toque_no_nulo]
  #print("Cantidad de contactados por SMS, WA que realizarón una solicitud:", len(filtro_sms_wa.SOLICITUD.unique()))
  SMS_WA_2 = len(filtro_sms_wa.SOLICITUD.unique())

  #Clientes contactados por SMS, WA que realizarón una solicitud, aperturaron otro producto
  mask_ap_other = (filtro_sms_wa['ID_ST'] == "AP") & (filtro_sms_wa['PRODUCTO'] != "Préstamo Directo Nómina") & (filtro_sms_wa['PRODUCTO'].notnull())
  #print("Cantidad de contactados por SMS, WA que realizarón una solicitud y aperturaron otro producto:", len(filtro_sms_wa[mask_ap_other]))
  SMS_WA_3 = len(filtro_sms_wa[mask_ap_other])

  #Clientes contactados por SMS, WA que realizarón una solicitud, aperturaron un prestamo directo de nomina
  mask_ap_pres = (filtro_sms_wa['ID_ST'] == "AP") & (filtro_sms_wa['PRODUCTO'] == "Préstamo Directo Nómina")
  filtro_sms_wa = filtro_sms_wa[mask_ap_pres]
  #print("Cantidad de contactados por SMS, WA que realizarón una solicitud y aperturaron un prestamo directo de nomina:", len(filtro_sms_wa))
  SMS_WA_4 = len(filtro_sms_wa)

  #Sumatoria de crédito solicitado:
  #print("Sumatoria de crédito solicitado:", filtro_sms_wa['CREDITO_SOLICITADO'].sum())
  SMS_WA_5 = filtro_sms_wa['CREDITO_SOLICITADO'].sum()

  #Tasa de solicitud
  SMS_WA_6 = round((SMS_WA_2 / SMS_WA_1 * 100),2)

  #Tasa de exito
  SMS_WA_7 = round((SMS_WA_4 / SMS_WA_1 * 100),2)

  #Vinculación post campaña
  SMS_WA_10 = round(((Suma_tenencia + SMS_WA_3 + SMS_WA_4)/ len(Tabla_tenencia)),3)
  #print("Vinculación post campaña",SMS_WA_10)

  return (SMS_WA_1, SMS_WA_2, SMS_WA_3, SMS_WA_4, SMS_WA_5, SMS_WA_6, SMS_WA_7, SMS_WA_8, SMS_WA_9, SMS_WA_10)

Canal SMS y MAIL

In [ ]:
def Canal_SMS_MAIL(toque):
  #Filtro para clientes contactados por SMS, EMAIL
  mask_sms_mail = dataset['CANAL'] == "SMS, EMAIL"
  filtro_sms_mail = dataset[mask_sms_mail]

  #Numero de contactados por SMS, EMAIL
  #print("Cantidad de contactados por SMS, EMAIL:", len(filtro_sms_mail.CLIENTE.unique()))
  SMS_MAIL_1 = len(filtro_sms_mail.CLIENTE.unique())

  #Tenecia por cliente
  Tabla_tenencia = filtro_sms_mail.loc[:,['CLIENTE','TENENCIA']]
  Tabla_tenencia = Tabla_tenencia.drop_duplicates(subset='CLIENTE')

  #Suma de tenencia por cliente
  Suma_tenencia = Tabla_tenencia['TENENCIA'].sum()
  SMS_MAIL_8 = Suma_tenencia

  #Vinculación pre campaña
  SMS_MAIL_9 = round((Suma_tenencia / len(Tabla_tenencia)),3)
  #print("Vinculación pre campaña",SMS_MAIL_9)

  #Contactados por SMS, EMAIL que ralizarón una solicitud
  mask_solic_no_nula = filtro_sms_mail['SOLICITUD'].notnull()
  if(toque == 1):
    mask_toque_no_nulo = filtro_sms_mail['TOQUE'].notnull()
  if(toque == 2):
    mask_toque_no_nulo = filtro_sms_mail['TOQUE'] == 1
  if(toque == 3):
    mask_toque_no_nulo = filtro_sms_mail['TOQUE'] == 2
  filtro_sms_mail = filtro_sms_mail[mask_solic_no_nula & mask_toque_no_nulo]
  #print("Cantidad de contactados por SMS, EMAIL que realizarón una solicitud:", len(filtro_sms_mail.SOLICITUD.unique()))
  SMS_MAIL_2 = len(filtro_sms_mail.SOLICITUD.unique())

  #Clientes contactados por SMS, EMAIL que realizarón una solicitud, aperturaron otro producto
  mask_ap_pres = (filtro_sms_mail['ID_ST'] == "AP") & (filtro_sms_mail['PRODUCTO'] != "Préstamo Directo Nómina") & (filtro_sms_mail['PRODUCTO'].notnull())
  #print("Cantidad de contactados por SMS, EMAIL que realizarón una solicitud y aperturaron otro producto:", len(filtro_sms_mail[mask_ap_pres]))
  SMS_MAIL_3 = len(filtro_sms_mail[mask_ap_pres])

  #Clientes contactados por SMS, EMAIL que realizarón una solicitud, aperturaron un prestamo directo de nomina
  mask_ap_pres = (filtro_sms_mail['ID_ST'] == "AP") & (filtro_sms_mail['PRODUCTO'] == "Préstamo Directo Nómina")
  filtro_sms_mail = filtro_sms_mail[mask_ap_pres]
  #print("Cantidad de contactados por SMS, EMAIL que realizarón una solicitud y aperturaron un prestamo directo de nomina:", len(filtro_sms_mail))
  SMS_MAIL_4 = len(filtro_sms_mail)

  #Sumatoria de crédito solicitado:
  #print("Sumatoria de crédito solicitado:", filtro_sms_mail['CREDITO_SOLICITADO'].sum())
  SMS_MAIL_5 = filtro_sms_mail['CREDITO_SOLICITADO'].sum()

  #Tasa de solicitud
  SMS_MAIL_6 = round((SMS_MAIL_2 / SMS_MAIL_1 * 100),2)

  #Tasa de exito
  SMS_MAIL_7 = round((SMS_MAIL_4 / SMS_MAIL_1 * 100),2)

  #Vinculación post campaña
  SMS_MAIL_10 = round(((Suma_tenencia + SMS_MAIL_3 + SMS_MAIL_4)/ len(Tabla_tenencia)),3)
  #print("Vinculación post campaña",SMS_MAIL_10)

  return (SMS_MAIL_1, SMS_MAIL_2, SMS_MAIL_3, SMS_MAIL_4, SMS_MAIL_5, SMS_MAIL_6, SMS_MAIL_7, SMS_MAIL_8, SMS_MAIL_9, SMS_MAIL_10)

Canal SMS, MAIL y WA

In [ ]:
def Canal_SMS_MAIL_WA(toque):
  #Filtro para clientes contactados por SMS, EMAIL, WA
  mask_sms_mail_wa = dataset['CANAL'] == "SMS, EMAIL, WA"
  filtro_sms_mail_wa = dataset[mask_sms_mail_wa]

  #Numero de contactados por SMS, EMAIL, WA
  #print("Cantidad de contactados por SMS, EMAIL, WA:", len(filtro_sms_mail_wa.CLIENTE.unique()))
  SMS_MAIL_WA_1 = len(filtro_sms_mail_wa.CLIENTE.unique())

  #Tenecia por cliente
  Tabla_tenencia = filtro_sms_mail_wa.loc[:,['CLIENTE','TENENCIA']]
  Tabla_tenencia = Tabla_tenencia.drop_duplicates(subset='CLIENTE')

  #Suma de tenencia por cliente
  Suma_tenencia = Tabla_tenencia['TENENCIA'].sum()
  SMS_MAIL_WA_8 = Suma_tenencia

  #Vinculación pre campaña
  SMS_MAIL_WA_9 = round((Suma_tenencia / len(Tabla_tenencia)),3)
  #print("Vinculación pre campaña",SMS_MAIL_WA_9)

  #Contactados por SMS, EMAIL, WA que ralizarón una solicitud
  mask_solic_no_nula = filtro_sms_mail_wa['SOLICITUD'].notnull()
  if(toque == 1):
    mask_toque_no_nulo = filtro_sms_mail_wa['TOQUE'].notnull()
  if(toque == 2):
    mask_toque_no_nulo = filtro_sms_mail_wa['TOQUE'] == 1
  if(toque == 3):
    mask_toque_no_nulo = filtro_sms_mail_wa['TOQUE'] == 2
  filtro_sms_mail_wa = filtro_sms_mail_wa[mask_solic_no_nula & mask_toque_no_nulo]
  #print("Cantidad de contactados por SMS, EMAIL, WA que realizarón una solicitud:", len(filtro_sms_mail_wa.SOLICITUD.unique()))
  SMS_MAIL_WA_2 = len(filtro_sms_mail_wa.SOLICITUD.unique())

  #Clientes contactados por SMS, EMAIL, WA que realizarón una solicitud, aperturaron otro producto
  mask_ap_other = (filtro_sms_mail_wa['ID_ST'] == "AP") & (filtro_sms_mail_wa['PRODUCTO'] != "Préstamo Directo Nómina") & (filtro_sms_mail_wa['PRODUCTO'].notnull())
  #print("Cantidad de contactados por SMS, EMAIL, WA que realizarón una solicitud y aperturaron otro producto:", len(filtro_sms_mail_wa[mask_ap_other]))
  SMS_MAIL_WA_3 = len(filtro_sms_mail_wa[mask_ap_other])

  #Clientes contactados por SMS, EMAIL, WA que realizarón una solicitud, aperturaron un prestamo directo de nomina
  mask_ap_pres = (filtro_sms_mail_wa['ID_ST'] == "AP") & (filtro_sms_mail_wa['PRODUCTO'] == "Préstamo Directo Nómina")
  filtro_sms_mail_wa = filtro_sms_mail_wa[mask_ap_pres]
  #print("Cantidad de contactados por SMS, EMAIL, WA que realizarón una solicitud y aperturaron un prestamo directo de nomina:", len(filtro_sms_mail_wa))
  SMS_MAIL_WA_4 = len(filtro_sms_mail_wa)

  #Sumatoria de crédito solicitado:
  #print("Sumatoria de crédito solicitado:", filtro_sms_mail_wa['CREDITO_SOLICITADO'].sum())
  SMS_MAIL_WA_5 = filtro_sms_mail_wa['CREDITO_SOLICITADO'].sum()

  #Tasa de solicitud
  SMS_MAIL_WA_6 = round((SMS_MAIL_WA_2 / SMS_MAIL_WA_1 * 100),2)

  #Tasa de exito
  SMS_MAIL_WA_7 = round((SMS_MAIL_WA_4 / SMS_MAIL_WA_1 * 100),2)

  #Vinculación post campaña
  SMS_MAIL_WA_10 = round(((Suma_tenencia + SMS_MAIL_WA_3 + SMS_MAIL_WA_4)/ len(Tabla_tenencia)),3)
  #print("Vinculación post campaña",SMS_MAIL_WA_10)

  return (SMS_MAIL_WA_1, SMS_MAIL_WA_2, SMS_MAIL_WA_3, SMS_MAIL_WA_4, SMS_MAIL_WA_5, SMS_MAIL_WA_6, SMS_MAIL_WA_7, SMS_MAIL_WA_8, SMS_MAIL_WA_9, SMS_MAIL_WA_10)


Canal MAIL

In [ ]:
def Canal_MAIL(toque):
  #Filtro para clientes contactados por EMAIL
  mask_mail = dataset['CANAL'] == "EMAIL"
  filtro_mail = dataset[mask_mail]

  #Numero de contactados por EMAIL
  #print("Cantidad de contactados por EMAIL:", len(filtro_mail.CLIENTE.unique()))
  MAIL_1 = len(filtro_mail.CLIENTE.unique())

  #Tenecia por cliente
  Tabla_tenencia = filtro_mail.loc[:,['CLIENTE','TENENCIA']]
  Tabla_tenencia = Tabla_tenencia.drop_duplicates(subset='CLIENTE')

  #Suma de tenencia por cliente
  Suma_tenencia = Tabla_tenencia['TENENCIA'].sum()
  MAIL_8 = Suma_tenencia

  #Vinculación pre campaña
  MAIL_9 = round((Suma_tenencia / len(Tabla_tenencia)),3)
  #print("Vinculación pre campaña",MAIL_9)

  #Contactados por EMAIL que ralizarón una solicitud
  mask_solic_no_nula = filtro_mail['SOLICITUD'].notnull()
  if(toque == 1):
    mask_toque_no_nulo = filtro_mail['TOQUE'].notnull()
  if(toque == 2):
    mask_toque_no_nulo = filtro_mail['TOQUE'] == 1
  if(toque == 3):
    mask_toque_no_nulo = filtro_mail['TOQUE'] == 2
  filtro_mail = filtro_mail[mask_solic_no_nula & mask_toque_no_nulo]
  #print("Cantidad de contactados por EMAIL que realizarón una solicitud:", len(filtro_mail.SOLICITUD.unique()))
  MAIL_2 = len(filtro_mail.SOLICITUD.unique())

  #Clientes contactados por EMAIL que realizarón una solicitud, aperturaron otro producto
  mask_ap_other = (filtro_mail['ID_ST'] == "AP") & (filtro_mail['PRODUCTO'] != "Préstamo Directo Nómina")& (filtro_mail['PRODUCTO'].notnull())
  #print("Cantidad de contactados por EMAIL que realizarón una solicitud y aperturaron otro producto:", len(filtro_mail[mask_ap_other]))
  MAIL_3 = len(filtro_mail[mask_ap_other])

  #Clientes contactados por EMAIL que realizarón una solicitud, aperturaron un prestamo directo de nomina
  mask_ap_pres = (filtro_mail['ID_ST'] == "AP") & (filtro_mail['PRODUCTO'] == "Préstamo Directo Nómina")
  filtro_mail = filtro_mail[mask_ap_pres]
  #print("Cantidad de contactados por EMAIL que realizarón una solicitud y aperturaron un prestamo directo de nomina:", len(filtro_mail))
  MAIL_4 = len(filtro_mail)

  #Sumatoria de crédito solicitado:
  #print("Sumatoria de crédito solicitado:", filtro_mail['CREDITO_SOLICITADO'].sum())
  MAIL_5 = filtro_mail['CREDITO_SOLICITADO'].sum()

  #Tasa de solicitud
  MAIL_6 = round((MAIL_2 / MAIL_1 * 100),2)

  #Tasa de exito
  MAIL_7 = round((MAIL_4 / MAIL_1 * 100),2)

  #Vinculación post campaña
  MAIL_10 = round(((Suma_tenencia + MAIL_3 + MAIL_4)/ len(Tabla_tenencia)),3)
  #print("Vinculación post campaña",MAIL_10)

  return (MAIL_1, MAIL_2, MAIL_3, MAIL_4, MAIL_5, MAIL_6, MAIL_7, MAIL_8, MAIL_9, MAIL_10)

Testigos

In [ ]:
def Testigos(toque):
  #Número de clientes testigos
  mask_testigos = dataset['PT'] == "Testigo"
  filtro_testigos = dataset[mask_testigos]

  #Conteo de ID distintos
  Num_testigos = filtro_testigos['CLIENTE'].unique()
  #print("Número de clientes testigo:", len(Num_testigos))
  TEST_1 = len(Num_testigos)

  #Tenecia por cliente
  Tabla_tenencia = filtro_testigos.loc[:,['CLIENTE','TENENCIA']]
  Tabla_tenencia = Tabla_tenencia.drop_duplicates(subset='CLIENTE')

  #Suma de tenencia por cliente
  Suma_tenencia = Tabla_tenencia['TENENCIA'].sum()
  TEST_8 = Suma_tenencia

  #Vinculación pre campaña
  TEST_9 = round((Suma_tenencia / len(Tabla_tenencia)),3)
  #print("Vinculación pre campaña",TEST_9)

  #Numero de solicitudes
  mask_solic_no_nula = filtro_testigos['SOLICITUD'].notnull()
  if(toque == 1):
    mask_toque_no_nulo = filtro_testigos['TOQUE'].notnull()
  if(toque == 2):
    mask_toque_no_nulo = filtro_testigos['TOQUE'] == 1
  if(toque == 3):
    mask_toque_no_nulo = filtro_testigos['TOQUE'] == 2

  filtro_testigos = filtro_testigos[mask_solic_no_nula & mask_toque_no_nulo]
  #print("Cantidad de solicitudes únicas por testigos que se realizarón:", len(filtro_testigos.SOLICITUD.unique()))
  TEST_2 = len(filtro_testigos.SOLICITUD.unique())

  #Clientes testigos que realizarón una solicitud, aperturaron un producto
  mask_ap_other = (filtro_testigos['ID_ST'] == "AP") & (filtro_testigos['PRODUCTO']!= "Préstamo Directo Nómina") & (filtro_testigos['PRODUCTO'].notnull())
  #print("Cantidad de testigos que realizarón una solicitud y aperturaron otro producto:", len(filtro_testigos[mask_ap_other]))
  TEST_3 = len(filtro_testigos[mask_ap_other])

  #Clientes testigos que realizarón una solicitud, aperturaron un prestamo directo de nomina
  mask_ap_pres = (filtro_testigos['ID_ST'] == "AP") & (filtro_testigos['PRODUCTO'] == "Préstamo Directo Nómina")
  filtro_testigos = filtro_testigos[mask_ap_pres]
  #print("Cantidad de testigos que realizarón una solicitud y aperturaron un prestamo directo de nomina:", len(filtro_testigos))
  TEST_4 = len(filtro_testigos)

  #Sumatoria de crédito solicitado:
  #print("Sumatoria de crédito solicitado:", filtro_testigos['CREDITO_SOLICITADO'].sum())
  TEST_5 = filtro_testigos['CREDITO_SOLICITADO'].sum()

  #Tasa de solicitud
  TEST_6 = round((TEST_2 / TEST_1 * 100),2)

  #Tasa de exito
  TEST_7 = round((TEST_4 / TEST_1 * 100),2)

  #Vinculación post campaña
  TEST_10 = round(((Suma_tenencia + TEST_3 + TEST_4)/ len(Tabla_tenencia)),3)
  #print("Vinculación post campaña",TEST_10)

  return (TEST_1, TEST_2, TEST_3, TEST_4, TEST_5, TEST_6, TEST_7, TEST_8, TEST_9, TEST_10)

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

def simulador_exito(sig_alpha, Poblacion_A, Exitos_A, Poblacion_B, Exitos_B):
  bandera = 1
  while bandera == 1:
      try:
          sig_alpha = float(sig_alpha) / 100
          if float(sig_alpha) == 0 or float(sig_alpha) > 0.2:
              print("valor no disponible")
          else:
              bandera = 0
      except:
          print("valor no disponible")

  bandera = 1
  while bandera == 1:
      try:
          Poblacion_A = float(Poblacion_A)
          if Poblacion_A <= 0 or (Poblacion_A - int(Poblacion_A)) > 0:
              print("valor no disponible")
          else:
              bandera = 0
      except:
          print("valor no disponible")

  bandera = 1
  while bandera == 1:
      try:
          Exitos_A = float(1.00)
          if Exitos_A < 0 or Exitos_A > Poblacion_A or (Exitos_A - int(Exitos_A)) > 0:
              print("valor no disponible")
          else:
              bandera = 0
      except:
          print("valor no disponible")

  bandera = 1
  while bandera == 1:
      try:
          Poblacion_B = float(Poblacion_B)
          if Poblacion_B < 0 or (Poblacion_B - int(Poblacion_B)) > 0:
              print("valor no disponible")
          else:
              bandera = 0
      except:
          print("valor no disponible")

  bandera = 1
  while bandera == 1:
      try:
          Exitos_B = float(Exitos_B)
          if Exitos_B < 0 or Exitos_B > Poblacion_B or (Exitos_B - int(Exitos_B)) > 0:
              print("valor no disponible")
          else:
              bandera = 0
      except:
          print("valor no disponible")

      valor = "3"
      if valor == "1":
          HipotesisHa = "two-sided"
      elif valor == "2":
          HipotesisHa = "smaller"
      elif valor == "3":
          HipotesisHa = "larger"

      tipo = "BAD"
      i = 0
      while tipo == "BAD":
          Exitos_A = Exitos_A + i
          PropA = round(Exitos_A / Poblacion_A, 4)
          PropB = round(Exitos_B / Poblacion_B, 4)
          Exitos = np.array([Exitos_A, Exitos_B])
          Poblaciones = np.array([Poblacion_A, Poblacion_B])
          VEstadistico, P_Valor = proportions_ztest(count=Exitos, nobs=Poblaciones, alternative=HipotesisHa)
          if P_Valor < float(sig_alpha):
              diferencial = (PropA * 100) - (PropB * 100)
              print("\n Para rechazar la hipótesis nula se debe considerar los siguientes valores: ")
              print("Un Diferencial: ", round(diferencial, 4), "% y una tasa de respuesta del grupo piloto: ",
                    PropA * 100, "%, esto implica ~", Exitos_A, " casos de éxito")
              tipo = "OK"
          else:
              tipo = "BAD"
          i = 1

  respuesta2 = input("""\n¿Quieres probar otros valores ?
                    1. Sí.
                    2. No.
                    3. Regresar al menú.
                    opción: """)
  if respuesta2 == "1":
      respuesta2 = "SI"
  elif respuesta2 == "2":
      respuesta2 = "NO"
  elif respuesta2 == "3":
      respuesta2 = "OTRO"

  return respuesta2



Generar CSV

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.cell import MergedCell

# Guardar el archivo Excel
with pd.ExcelWriter('output.xlsx', engine='openpyxl') as writer:
    workbook = writer.book
    worksheet = workbook.create_sheet("Sheet1")

    # Estilos generales
    header_font = Font(bold=True, color="FFFFFF", size=12)
    header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")
    border_style = Border(
        left=Side(style="thin"), right=Side(style="thin"),
        top=Side(style="thin"), bottom=Side(style="thin")
    )

    # Bucle para las tres tablas
    for i, toque in enumerate([1, 2, 3]):
        # Obtener los datos
        (SMS_1, SMS_2, SMS_3, SMS_4, SMS_5, SMS_6, SMS_7, SMS_8, SMS_9, SMS_10) = Canal_SMS(toque)
        (SMS_WA_1, SMS_WA_2, SMS_WA_3, SMS_WA_4, SMS_WA_5, SMS_WA_6, SMS_WA_7, SMS_WA_8, SMS_WA_9, SMS_WA_10) = Canal_SMS_WA(toque)
        (SMS_MAIL_1, SMS_MAIL_2, SMS_MAIL_3, SMS_MAIL_4, SMS_MAIL_5, SMS_MAIL_6, SMS_MAIL_7, SMS_MAIL_8, SMS_MAIL_9, SMS_MAIL_10) = Canal_SMS_MAIL(toque)
        (SMS_MAIL_WA_1, SMS_MAIL_WA_2, SMS_MAIL_WA_3, SMS_MAIL_WA_4, SMS_MAIL_WA_5, SMS_MAIL_WA_6, SMS_MAIL_WA_7, SMS_MAIL_WA_8, SMS_MAIL_WA_9, SMS_MAIL_WA_10) = Canal_SMS_MAIL_WA(toque)
        (MAIL_1, MAIL_2, MAIL_3, MAIL_4, MAIL_5, MAIL_6, MAIL_7, MAIL_8, MAIL_9, MAIL_10) = Canal_MAIL(toque)
        (TEST_1, TEST_2, TEST_3, TEST_4, TEST_5, TEST_6, TEST_7, TEST_8, TEST_9, TEST_10) = Testigos(toque)

        #Calcular totales
        SUM_1 = SMS_1 + SMS_WA_1 + SMS_MAIL_1 + SMS_MAIL_WA_1 + MAIL_1
        SUM_2 = SMS_2 + SMS_WA_2 + SMS_MAIL_2 + SMS_MAIL_WA_2 + MAIL_2
        SUM_3 = SMS_3 + SMS_WA_3 + SMS_MAIL_3 + SMS_MAIL_WA_3 + MAIL_3
        SUM_4 = SMS_4 + SMS_WA_4 + SMS_MAIL_4 + SMS_MAIL_WA_4 + MAIL_4
        SUM_5 = SMS_5 + SMS_WA_5 + SMS_MAIL_5 + SMS_MAIL_WA_5 + MAIL_5
        SUM_8 = SMS_8 + SMS_WA_8 + SMS_MAIL_8 + SMS_MAIL_WA_8 + MAIL_8

        #######################################################################
        print("_______________________________________________________________")
        simulador_exito(0.01,SUM_1,1,TEST_1,TEST_4)
        print("Usados")
        print ("total piloto", SUM_1)
        print ("exitos piloto", 1)
        print ("total testigo", TEST_1)
        print ("exitos testigo", TEST_4)
        print("_______________________________________________________________")
        #######################################################################

        # Crear el DataFrame para cada tabla
        data = {
            "GRUPO (PILOTO O TESTIGO)": ["PILOTO", "", "", "", "", "TOTAL", "TESTIGO"],
            "CANAL": ["SMS", "SMS_WA", "SMS_EMAIL", "SMS_EMAIL_WA", "EMAIL", "", None],
            "CLIENTES": [SMS_1, SMS_WA_1, SMS_MAIL_1, SMS_MAIL_WA_1, MAIL_1, SUM_1, TEST_1],
            "SOLICITUDES": [SMS_2, SMS_WA_2, SMS_MAIL_2, SMS_MAIL_WA_2, MAIL_2, SUM_2, TEST_2],
            "EXITO OP": [SMS_3, SMS_WA_3, SMS_MAIL_3, SMS_MAIL_WA_3, MAIL_3, SUM_3, TEST_3],
            "EXITO PO": [SMS_4, SMS_WA_4, SMS_MAIL_4, SMS_MAIL_WA_4, MAIL_4, SUM_4, TEST_4],
            "TASA DE SOLS": [
                str(SMS_6) + "%",
                str(SMS_WA_6) + "%",
                str(SMS_MAIL_6) + "%",
                str(SMS_MAIL_WA_6) + "%",
                str(MAIL_6) + "%",
                str(round((SUM_2 / SUM_1 * 100),2)) + "%",
                str(TEST_6) + "%"
            ],
            "TASA DE ÉXITO": [
                str(SMS_7) + "%",
                str(SMS_WA_7) + "%",
                str(SMS_MAIL_7) + "%",
                str(SMS_MAIL_WA_7) + "%",
                str(MAIL_7) + "%",
                str(round((SUM_4 / SUM_1 * 100),2)) + "%",
                str(TEST_7) + "%"
            ],
            "TENENCIA PREVIA": [SMS_8, SMS_WA_8, SMS_MAIL_8, SMS_MAIL_WA_8, MAIL_8, SUM_8, TEST_8],
            "VINCULACIÓN PREVIA": [SMS_9, SMS_WA_9, SMS_MAIL_9, SMS_MAIL_WA_9, MAIL_9, round((SUM_8/SUM_1),3) , TEST_9],
            "VINCULACIÓN NUEVA": [SMS_10, SMS_WA_10, SMS_MAIL_10, SMS_MAIL_WA_10, MAIL_10, round(((SUM_8+SUM_3+SUM_4)/SUM_1),3) , TEST_10],
            "LINEA DE CRÉDITO TOTAL": [SMS_5, SMS_WA_5, SMS_MAIL_5, SMS_MAIL_WA_5, MAIL_5, SUM_5, TEST_5]
        }
        df = pd.DataFrame(data)

        # Calcular inicio de la tabla
        start_row = i * (len(df) + 4) + 1
        start_column = 2  # Mover una celda a la derecha (columna B)

        # Título de la tabla
        worksheet.merge_cells(
            start_row=start_row,
            start_column=start_column,
            end_row=start_row,
            end_column=start_column + len(df.columns) - 1
        )
        title_cell = worksheet.cell(row=start_row, column=start_column)
        if(toque == 1):
          title_cell.value = f"RESULTADOS TOQUE 1 Y 2"
        if(toque == 2):
          title_cell.value = f"RESULTADOS TOQUE 1"
        if(toque == 3):
          title_cell.value = f"RESULTADOS TOQUE 2"
        title_cell.font = Font(bold=True, size=14)
        title_cell.alignment = Alignment(horizontal="center")

        # Escribir DataFrame
        for row_idx, row_data in enumerate(df.itertuples(index=False), start=start_row + 2):
            for col_idx, value in enumerate(row_data, start=start_column):
                cell = worksheet.cell(row=row_idx, column=col_idx)
                cell.value = value
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.border = border_style

        # Encabezados con estilos
        header_row = start_row + 1
        for col_idx, header in enumerate(df.columns, start=start_column):
            cell = worksheet.cell(row=header_row, column=col_idx)
            cell.value = header
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = border_style

        # Ajustar ancho de columnas
        from openpyxl.utils import get_column_letter
        for col_idx in range(start_column, start_column + len(df.columns)):
            column_letter = get_column_letter(col_idx)
            max_length = max(
                (len(str(worksheet.cell(row=row_idx, column=col_idx).value)) for row_idx in range(1, worksheet.max_row + 1)),
                default=0
            )
            worksheet.column_dimensions[column_letter].width = max_length + 5

print("Archivo 'output.xlsx' guardado.")

_______________________________________________________________

 Para rechazar la hipótesis nula se debe considerar los siguientes valores: 
Un Diferencial:  0.7 % y una tasa de respuesta del grupo piloto:  3.19 %, esto implica ~ 1748.0  casos de éxito

¿Quieres probar otros valores ? 
                    1. Sí.
                    2. No.
                    3. Regresar al menú.
                    opción: no
Usados
total piloto 54836
exitos piloto 1
total testigo 10112
exitos testigo 252
_______________________________________________________________
_______________________________________________________________

 Para rechazar la hipótesis nula se debe considerar los siguientes valores: 
Un Diferencial:  0.46 % y una tasa de respuesta del grupo piloto:  1.4200000000000002 %, esto implica ~ 781.0  casos de éxito

¿Quieres probar otros valores ? 
                    1. Sí.
                    2. No.
                    3. Regresar al menú.
                    opción: no
Usados
total 